In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown

### Refresh dot env

In [ ]:
load_dotenv(override=True)

In [3]:
open_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

### Create initial query to get challange reccomendation

In [4]:
query = 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. '
query += 'Answer only with the question, no explanation.'

messages = [{'role':'user', 'content':query}]

In [ ]:
print(messages)

### Call openai gpt-4o-mini 

In [6]:
openai = OpenAI()

response = openai.chat.completions.create(
    messages=messages,
    model='gpt-4o-mini'
)

challange = response.choices[0].message.content


In [ ]:
print(challange)

In [8]:
competitors = []
answers = []

### Create messages with the challange query

In [9]:
messages = [{'role':'user', 'content':challange}]

In [ ]:
print(messages)

In [ ]:
!ollama pull llama3.2

In [12]:
from threading import Thread

In [13]:
def gpt_mini_processor():
    modleName = 'gpt-4o-mini'
    competitors.append(modleName)
    response_gpt = openai.chat.completions.create(
        messages=messages,
        model=modleName
    )
    answers.append(response_gpt.choices[0].message.content)

def gemini_processor():
    gemini = OpenAI(api_key=google_api_key, base_url='https://generativelanguage.googleapis.com/v1beta/openai/')
    modleName = 'gemini-2.0-flash'
    competitors.append(modleName)
    response_gemini = gemini.chat.completions.create(
        messages=messages,
        model=modleName
    )
    answers.append(response_gemini.choices[0].message.content)

def llama_processor():
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    modleName = 'llama3.2'
    competitors.append(modleName)
    response_llama = ollama.chat.completions.create(
        messages=messages,
        model=modleName
    )
    answers.append(response_llama.choices[0].message.content)

### Paraller execution of LLM calls

In [14]:
thread1 = Thread(target=gpt_mini_processor)
thread2 = Thread(target=gemini_processor)
thread3 = Thread(target=llama_processor)

thread1.start()
thread2.start()
thread3.start()

thread1.join()
thread2.join()
thread3.join()

In [ ]:
print(competitors)
print(answers)

In [ ]:
for competitor, answer in zip(competitors, answers):
    print(f'Competitor:{competitor}\n\n{answer}')

In [17]:
together = ''
for index, answer in enumerate(answers):
    together += f'# Response from competitor {index + 1}\n\n'
    together += answer + '\n\n'

In [ ]:
print(together)

### Prompt to judge the LLM results

In [19]:
to_judge = f'''You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{challange}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""

'''

In [20]:
to_judge_message = [{'role':'user', 'content':to_judge}]

### Execute o3-mini to analyze the LLM results

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    messages=to_judge_message,
    model='o3-mini'
)
result = response.choices[0].message.content
print(result)

In [ ]:
results_dict = json.loads(result)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")